In [ ]:
# %cd ./work
# %pwd

In [ ]:
%run "../catalog_common.py"

In [ ]:
#preamble to analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import gca, mpl

%matplotlib inline
import seaborn as sns
import matplotlib.ticker
from IPython.display import Markdown as md
from IPython.display import HTML, display
from time import sleep

from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt
opt.order = []  # no sorting

In [ ]:
alldf = pd.read_csv('operator.csv',low_memory=False)
alldf.date = pd.to_datetime(alldf.date)
alldf['year'] = alldf.date.dt.year
alldf = alldf[alldf.year>2010]  # don't include historic data
opname = alldf.bgOperatorName.iloc[0]
commname = alldf.OperatorName.value_counts().index[0]

In [ ]:
ID_header(opname, line2 = commname,
          subtitle='Open-FF Operator Summary',incl_links=True,
          link_up_level=True)
set_page_param()

In [ ]:
disc_w_ing = alldf.groupby('UploadKey')['ingKeyPresent'].first().sum()
disc_w_water = len(alldf[alldf.TotalBaseWaterVolume>0].UploadKey.unique())
display(md(f'### Total number of disclosures : {len(alldf.UploadKey.unique()):,}'))
display(md(f'### Number of disclosures with water volume: {disc_w_water:,}'))
display(md(f'### Number of disclosures with chemical records: {disc_w_ing:,}'))


In [ ]:
display(md(f'## Disclosure locations: {opname.upper()}'))

In [ ]:
def xlate_val(n):
    if n==0:
        return ''
    if n<1000:
        return round_sig(n,1)
    x = round_sig(n,1)
    return x[0]+ 'k'

def make_annot(gb):
    annot = gb.copy()
    annot.UploadKey = annot.UploadKey.map(lambda x: xlate_val(x))
    #print(annot)
    piv = annot.pivot(index='County',columns='year',values='UploadKey')
    piv.fillna('',inplace=True)
    #print(piv)
    return piv
    
def CountyMap(df):
    state_list = df.bgStateName.unique().tolist()
    start_loc = get_geog_center(state_list)
    #print(statename,start_loc)
    cond = (df.loc_within_state=='YES')&(df.loc_within_county=='YES')
    if cond.sum()==0:  # no valid fracks for this state
        display(md('## No mappable fracks for this operator!'))
        # display(md(f'Any data in this state set may be labeled incorrectly as {statename}'))
        return
    gb = df[cond].groupby(['bgStateName','bgCountyName',
                                                   'UploadKey'],as_index=False)['bgCAS'].count()
    gb = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['UploadKey'].count().rename({'bgStateName':'StateName',
                                                                                                'bgCountyName':'CountyName',
                                                                                                'UploadKey':'value'},
                                                                                                axis=1)
    zoom = get_zoom_level(df[['bgLatitude','bgLongitude']])
    # if statename in ['texas','california']:
    #     zoom = 5
    # if statename in ['alaska']:
    #     zoom = 4
    # Look here for different way to scale appropriately:
    # https://stackoverflow.com/questions/58162200/pre-determine-optimal-level-of-zoom-in-folium
    
    # zoom = 3.5
    create_county_choropleth(gb,plotlog=False,#plotlog=True,custom_scale= [0,1,2,3,4],
                             start_loc=start_loc, # center of state's data
                             legend_name='Number of FracFocus disclosures',
                             start_zoom=zoom,fields=['StateName','CountyName','orig_value'],
                             aliases=['State: ','County: ','Number Fracking disclosures: '])

def PointMap(df):
    gb = df.groupby('UploadKey')[['bgLatitude','bgLongitude','APINumber','TotalBaseWaterVolume',
                                 'year','OperatorName','ingKeyPresent']].first()
    gb['year'] = gb.year.astype('str')
    gb.TotalBaseWaterVolume = gb.TotalBaseWaterVolume.map(lambda x: round_sig(x,3,guarantee_str='??')) + ' gallons'
    gb.APINumber = gb.APINumber.astype('str')
    # gb.drop('date',axis=1,inplace=True)
    create_point_map(gb)
    
    

### ...by county
Note that not all fracks have valid County information and so will not show up in the following map (especially Alaska).

In [ ]:
CountyMap(alldf)

### ...by individual wells
Use the layer controls in the upper right to turn on and off the types of markers as well as showing the map or satellite views.  Cluster markers are best for big picture view, information markers include popups with a bit of info about the well.  Note that for operators with a large number of disclosures, the information markers take a few seconds to update.

In [ ]:
def make_disc_link(row):
    return getDisclosureLink(row.APINumber,row.UploadKey,row.APINumber)

alldf.APINumber = alldf.apply(lambda x: make_disc_link(x),axis=1)

PointMap(alldf)

# Active Years by state

In [ ]:
gb = alldf.groupby(['bgStateName','UploadKey','year'],as_index=False).size()
gb1 = gb.groupby(['bgStateName','year'],as_index=False).size().rename({'size':'num_disc'},axis=1)
gb1.bgStateName = gb1.bgStateName.str.title()
nstates = len(gb1.bgStateName.unique())
dim = 4
if nstates>12:
    dim = 5
if nstates>15:
    dim = 6
pd.pivot_table(gb1,index='year',columns='bgStateName',values='num_disc').plot.bar(subplots=True,
                                                                                 layout=(dim,3),
                                                                                 legend=False,
                                                                                 figsize=(10,10),
                                                                                 title='Number of disclosures');
# for state in states:
#     gb1[gb1.bgStateName==state].plot.bar('year','num_disc',title=f'{state.title()}',figsize=(3,2),
#                                          ylabel='Number of disclosures',legend=False)

# Chemicals of Concern
Percent of this company's disclosures with chemicals on the following lists:
- Clean Water Act list
- Safe Drinking Water Act list
- California's Prop. 65 list
- TSCA's "UVCB" list - "Unknown, variable composition..."
- EPA's PFAS master list
- EPA's diesel list (4 chemicals)

In [ ]:
gb= alldf.groupby('UploadKey',as_index=False)['is_on_CWA','is_on_DWSHA','is_on_PFAS_list',
                                             'is_on_prop65',
                                             'is_on_UVCB','is_on_diesel'].sum()
CWA = len(gb[gb.is_on_CWA>0])/disc_w_ing *100
DWSHA = len(gb[gb.is_on_DWSHA>0])/disc_w_ing *100
PFAS = len(gb[gb.is_on_PFAS_list>0])/disc_w_ing *100
# volatile = len(gb[gb.is_on_volatile_list>0])/disc_w_ing *100
prop65 = len(gb[gb.is_on_prop65>0])/disc_w_ing *100
UVCB = len(gb[gb.is_on_UVCB>0])/disc_w_ing *100
diesel = len(gb[gb.is_on_diesel>0])/disc_w_ing *100
t = pd.DataFrame({'list':['CWA','DWSHA','prop65','UVCB','PFAS','diesel'],
                  'Percent_disc':[CWA,DWSHA,prop65,UVCB,PFAS,diesel]})
ax = t.plot.bar('list','Percent_disc',legend=False,ylabel='% of disclosures with at least\n one chemical on list')
plt.xticks(rotation=0);

# Water use
Reported `TotalBaseWaterVolume` on the disclosure (in gallons).

In [ ]:
totpres = len(alldf.UploadKey.unique())

if totpres<300:
    alpha = 1
elif totpres<2000:
    alpha = .6
elif totpres<20000:
    alpha = .35
else:
    alpha = .2
    
min_water = 10 # number disclosures needed to trigger graphs

In [ ]:
if (disc_w_water>min_water):
    gb = alldf.groupby('UploadKey',as_index=False)[['TotalBaseWaterVolume','date']].first()
    cond = (gb.TotalBaseWaterVolume>0) & (gb.date.dt.year>2010)
    gb['not_present'] = gb[cond].TotalBaseWaterVolume.min() - (gb[cond].TotalBaseWaterVolume.max()-gb[cond].TotalBaseWaterVolume.min())*0.05
    ax = gb[cond].plot('date','TotalBaseWaterVolume', style='o', alpha=alpha,
                figsize=(16,6),legend=False)
    gb[~cond].plot('date','not_present', style='|', alpha=1,color='orange',ms=20, ax=ax,legend=False)
    plt.ylabel('Gallons reported',fontsize=16);
    plt.title(f'Gallons of water used for each job: {opname.upper()} -- linear version',fontsize=16);
    ax.grid()
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    ax = gca().yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'));
    display(md(f'##### Number of disclosures with valid water volume (shown by blue circles): {len(gb[cond]):,}\n'))
    display(md(f'##### Number of records without valid water volume (shown by orange bars): {len(gb[~cond]):,}\n'))


In [ ]:
# if alldf.TotalBaseWaterVolume.max()>0:
#     display(md(f'##### Number of disclosures with valid water volume (shown by blue circles): {len(gb[cond]):,}\n'))
#     display(md(f'##### Number of records without valid water volume (shown by orange bars): {len(gb[~cond]):,}\n'))
#     display(md('---'))

In [ ]:
if (disc_w_water>min_water):
    ax = gb[cond].plot('date','TotalBaseWaterVolume', style='o', alpha=alpha,
                figsize=(16,6))
    plt.ylabel('Gallons reported',fontsize=16);
    plt.title(f'Gallons of water used for each job: {opname.upper()} -- log version',fontsize=16);
    ax.set(yscale='log')
    ax.tick_params(axis="y", labelsize=14)
    ax.tick_params(axis="x", labelsize=14)
    if len(gb[cond]) < 5000: # provide more detailed grid (too many points swamps it out)
        locmaj = matplotlib.ticker.LogLocator(base=10,subs='all') 
    else:
        locmaj = matplotlib.ticker.LogLocator(base=10) #,subs='all') 
    ax.yaxis.set_major_locator(locmaj)
    ax.set(ylim=(max(10,gb.TotalBaseWaterVolume.min()),
                 gb.TotalBaseWaterVolume.max()*1.4));
    ax.grid()

    lns = list(np.percentile(gb[cond].TotalBaseWaterVolume,[25,50,75]))
    #ax.set_ylim(-0.7,len(sn)-0.3)
    for l in lns:
        plt.hlines(l,gb[cond].date.min(),
                   gb[cond].date.max(),
                   color='black')
    s = 'PERCENTILES:\n'
    s+= ' -- 25%:  {:,} gallons\n'.format(float(lns[0]))
    s+= ' -- 50%:  {:,} gallons\n'.format(float(lns[1]))
    s+= ' -- 75%:  {:,} gallons\n'.format(float(lns[2]))
    s+= ' -- max:  {:,} gallons\n'.format(float(gb[cond].TotalBaseWaterVolume.max()))
    print(s)

In [ ]:
if (disc_w_water<=min_water)&(alldf.TotalBaseWaterVolume.max()>0):
    gb = alldf[alldf.TotalBaseWaterVolume>0].groupby('UploadKey',as_index=False)[['date','APINumber',
                                                                   'TotalBaseWaterVolume',
                                                                   'bgStateName','bgCountyName']].first()
    iShow(gb[['date','APINumber','TotalBaseWaterVolume','bgStateName','bgCountyName']])

# Suppliers/Service companies used by this Operator

In [ ]:
if disc_w_ing>0:
    gb1 = alldf.groupby('bgSupplier',as_index=False)[['UploadKey']].nunique().rename({'UploadKey':'disclosure_cnt'},axis=1)
    # gb1 = gb.groupby('bgSupplier',as_index=False)['UploadKey'].count()
    gb2 = alldf.groupby('bgSupplier')['Supplier'].agg(lambda x: x.value_counts().index[0])
    gb2 = gb2.reset_index()
    gb2.columns = ['bgSupplier','most_common_name']
    mg = pd.merge(gb2,gb1,on='bgSupplier',how='outer')
    iShow(mg)
else:
    display(md('>### **No disclosures with chemical records for this operator**'))

In [ ]:
from matplotlib.offsetbox import AnchoredText


def proprietary_plot(df,plot_title='TEST',minyr=2011,maxyr=2021):
    df = df.copy()
    df['year'] = df.date.dt.year
    df = df[df.year<=maxyr]
    df = df[df.year>=minyr]
    prop = df.bgCAS=='proprietary'
    gb = df[prop].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
    gb1 = df[df.is_valid_cas].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
    gb2 = df.groupby('UploadKey',as_index=False)['date'].first()
    mg = pd.merge(gb2,gb,on='UploadKey',how='left')
    mg = pd.merge(mg,gb1,on='UploadKey',how='left')
    mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
    mg['percProp'] = (mg.numprop / mg.numvalid) * 100

    mg['propCut'] = pd.cut(mg.percProp,right=False,bins=[0,0.0001,10,25,50,101],
                          labels=['no proprietary claims','up to 10% proprietary claims',
                                  'between 10 and 25% proprietary claims',
                                  'between 25 and 50% proprietary claims',
                                  'greater than 50% proprietary claims'])
    mg['year'] = mg.date.dt.year
    out = mg.drop(['date','UploadKey'],axis=1)
    t = out[out.numvalid>0].groupby(['year','propCut'],as_index=False)['numvalid'].count()
    sums = t.groupby('year',as_index=False)['numvalid'].sum().rename({'numvalid':'tot'},axis=1)
    t = pd.merge(t,sums,on='year',how='left')
    t['PercentProp'] = t.numvalid/t.tot *100

    piv = t.pivot(index='year', columns='propCut', values='PercentProp')

    ax = piv.plot.area(figsize=(12,7),ylim=(0,100),xlim=(minyr,maxyr),colormap='Reds')
    ax.set_title(f'Percentage of valid records that are Trade Secret claims at the disclosure level', fontsize=16)
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], title='Disclosure Proprietary\nPercentage class\n',
              loc='upper left',bbox_to_anchor=(1, 1))
    ax.set_ylabel('Percentage of disclosures', fontsize=16)
    ax.set_xlabel('Year', fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.suptitle(f'{plot_title}',fontsize=24)

    gb = df.groupby(['year','UploadKey'],as_index=False)['bgCAS'].count()
    gb = gb.groupby('year',as_index=False)['UploadKey'].count()#.rename({'UploadKey':'number of disclosures'},axis=1)
    s = 'Number of disclosures by year:\n\n'
    for i,row in gb.iterrows():
        s+= f'   {row.year}: {row.UploadKey:7,} \n'
    at2 = AnchoredText(s,
                       loc='lower left', prop=dict(size=10), frameon=False,
                       bbox_to_anchor=(1., 0.),
                       bbox_transform=ax.transAxes
                       )
    at2.patch.set_boxstyle("square,pad=0.")
    ax.add_artist(at2)

def proprietary_bars(df,plot_title='TEST',minyr=2011,maxyr=2021):
    df = df.copy()
    # print(df.columns)
    df['year'] = df.date.dt.year
    df = df[df.year<=maxyr]
    df = df[df.year>=minyr]
    prop = df.bgCAS=='proprietary'
    gb = df[prop].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numprop'},axis=1)
    gb1 = df[df.is_valid_cas].groupby('UploadKey',as_index=False)['bgCAS'].count().rename({'bgCAS':'numvalid'},axis=1)
    gb2 = df.groupby('UploadKey',as_index=False)['date'].first()
    mg = pd.merge(gb2,gb,on='UploadKey',how='left')
    mg = pd.merge(mg,gb1,on='UploadKey',how='left')
    mg.fillna(0,inplace=True) # there will be disclosures with 0 proprietary; need to fill
    mg['percProp'] = (mg.numprop / mg.numvalid) * 100

    mg['propCut'] = pd.cut(mg.percProp,right=False,bins=[0,0.0001,10,25,50,101],
                          labels=['no proprietary claims','up to 10% \nare proprietary claims',
                                  'between 10 and 25% \nare proprietary claims',
                                  'between 25 and 50% \nare proprietary claims',
                                  'greater than 50% \nare proprietary claims'])
    
    # mg.propCut.value_counts(sort=False).plot(kind='barh',colormap='Reds')
    # print(mg)
    import seaborn as sns
    t = mg.propCut.value_counts(sort=False).reset_index()
    # print(t)
    ax = sns.barplot(data=t,y='index',x='propCut',palette='Reds',orient="h")
    ax.set_xlabel("Number of disclosures")
    ax.set_ylabel("")
    ax.set_title(plot_title)
    ax.invert_yaxis()


if disc_w_ing > 0:
    display(md("# Proprietary claims"))
    testtitle = opname.upper() +': Trade Secret frequency'
    proprietary_bars(alldf,testtitle,
                     minyr=max(2011,alldf.date.dt.year.min()),
                     maxyr=alldf.date.dt.year.max())
